# Test Benchmark

## Buffer

buffer store function (multi done scenario)

In [1]:
from replay_buffer import ReplayBuffer
from agent import AgentSAC
from attrdict import AttrDict
from envs import ReachToyEnv
import torch
args = AttrDict(learning_rate=0.01, env_num=2, eval_gap=0, eval_steps_per_env=10, cwd='')
agent = AgentSAC(10,4,2,max_env_step=5, goal_dim=2,gpu_id=-1, info_dim=4+4,args=args)
env = ReachToyEnv(max_step=5, gpu_id=-1)
buffer = ReplayBuffer(gpu_id=-1,
                      max_len=10,
                      state_dim=4,
                      action_dim=2,
                      goal_dim=2,
                      reward_fn = env.compute_reward,
                      info_dim=4+4) # add traj index

/home/reed/miniconda3/envs/rlgpu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/reed/miniconda3/envs/rlgpu/lib/python3.7/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/reed/miniconda3/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
traj = agent.explore_env(env, target_steps_per_env=6, random_mode=True, buffer = buffer)

In [3]:
buffer.buf_state, buffer.buf_other[...,-1], buffer.buf_other[...,4+2:4+4]

(tensor([[-0.5454,  0.5129,  0.4291, -0.8733],
         [-0.6686,  0.4724,  0.4291, -0.8733],
         [-0.8177,  0.5388,  0.4291, -0.8733],
         [-0.6315,  0.7388,  0.4291, -0.8733],
         [-0.8315,  0.9388,  0.4291, -0.8733],
         [ 0.4646,  0.5766,  0.1615, -0.1874],
         [ 0.4359,  0.6360,  0.1615, -0.1874],
         [ 0.4410,  0.4360,  0.1615, -0.1874],
         [ 0.4089,  0.3468,  0.1615, -0.1874],
         [ 0.6089,  0.4523,  0.1615, -0.1874]]),
 tensor([4., 3., 2., 1., 0., 4., 3., 2., 1., 0.]),
 tensor([[-0.6686,  0.4724],
         [-0.8177,  0.5388],
         [-0.6315,  0.7388],
         [-0.8315,  0.9388],
         [-1.0000,  0.7559],
         [ 0.4359,  0.6360],
         [ 0.4410,  0.4360],
         [ 0.4089,  0.3468],
         [ 0.6089,  0.4523],
         [ 0.7106,  0.6523]]))

In [4]:
buffer.update_now_len()
rew, mask, act, s, ns, i= buffer.sample_batch(10, her_rate=1, indices=torch.arange(start=0,end=5))

In [6]:
s, rew, ns

(tensor([[-0.5454,  0.5129, -0.6686,  0.4724],
         [-0.6686,  0.4724, -0.8177,  0.5388],
         [-0.8177,  0.5388, -0.6315,  0.7388],
         [-0.6315,  0.7388, -0.8315,  0.9388]]),
 tensor([[-0.],
         [-0.],
         [-0.],
         [-0.]]),
 tensor([[-0.6686,  0.4724, -0.6686,  0.4724],
         [-0.8177,  0.5388, -0.8177,  0.5388],
         [-0.6315,  0.7388, -0.6315,  0.7388],
         [-0.8315,  0.9388, -0.8315,  0.9388]]))

HER buffer

In [ ]:
buffer.buf_other[:20][..., :2]